In [19]:
from sklearn.datasets import load_wine
from datetime import timedelta
import pandas as pd
import numpy as np

In [20]:
read_data_file = pd.read_pickle('../hive.pkl')
df = pd.DataFrame(read_data_file)

In [21]:
df['cal_time'][0:20].median()

Timedelta('15 days 00:43:00')

In [22]:
df['cal_time_binaly'] = df['cal_time'].apply(lambda x:  0 if x < timedelta (days = 15) else 1)

In [23]:
X = df[['commit_befor ', 'commit_after', 'cal_smell', 'dispen_bf', 'object_bf',
       'bloater_bf', 'dispen_at', 'object_at', 'bloater_af', 'open_time',
       'closed_time','changed_file', 'additions', 'deletions',
       'dev']]
y = df['cal_time_binaly']

In [24]:
from sklearn.ensemble import GradientBoostingClassifier
X_train, X_test = X[:20], X[20:]
y_train, y_test = y[:20], y[20:]
clf = GradientBoostingClassifier(n_estimators=1, learning_rate=1.0 , max_depth=1 , random_state=2).fit(X_train , y_train)

In [25]:
from sklearn.model_selection import cross_val_score
cross_val_score(clf , X , y , cv=5 , scoring= "accuracy")

array([0.83333333, 0.5       , 0.5       , 0.5       , 0.66666667])

In [26]:
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_predict
y_pred = cross_val_predict(clf,X,y,cv=5)
y_pred

array([1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1], dtype=int64)

In [27]:
clf.predict(X_test)

array([1, 1, 1, 1, 1, 1, 0, 1, 1, 1], dtype=int64)

In [28]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y , y_pred)

array([[ 3, 10],
       [ 2, 15]], dtype=int64)

In [29]:
from sklearn.metrics import precision_score, recall_score, f1_score
print('precision = ', precision_score(y,y_pred ,average='macro'))
print('recall_score = ', recall_score(y,y_pred ,average='macro'))
print('f1_score',f1_score(y,y_pred ,average='macro'))


precision =  0.6
recall_score =  0.5565610859728507
f1_score 0.5238095238095238


In [30]:
from sklearn.model_selection import GridSearchCV

parameters = {
  
    'learning_rate': [1.0,2.0],
    'n_estimators': [1,2,3],
    'subsample' : [0.7],
    'max_depth' : [1,2] ,
    'random_state':[2,3]
    #n_estimators=1, learning_rate=1.0 , max_depth=1 , random_state=2
  
}
GradientBoosting = GradientBoostingClassifier()
clf = GridSearchCV(GradientBoosting, parameters , cv=3 , scoring='f1_macro')
clf.fit(X_train,y_train)

GridSearchCV(cv=3, estimator=GradientBoostingClassifier(),
             param_grid={'learning_rate': [1.0, 2.0], 'max_depth': [1, 2],
                         'n_estimators': [1, 2, 3], 'random_state': [2, 3],
                         'subsample': [0.7]},
             scoring='f1_macro')

In [31]:
y_pred = cross_val_predict(clf.best_estimator_,X_test,y_test)

C:\Users\Beauz\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


In [32]:
clf.best_estimator_

GradientBoostingClassifier(learning_rate=1.0, max_depth=2, n_estimators=2,
                           random_state=3, subsample=0.7)

In [33]:
y_pred_clf = clf.best_estimator_.predict(X_test)

In [34]:
from sklearn.metrics import precision_score, recall_score, f1_score
print('precision = ', precision_score(y_test,y_pred_clf ))
print('recall_score = ', recall_score(y_test,y_pred_clf ))
print('f1_score',f1_score(y_test,y_pred_clf ))

precision =  0.625
recall_score =  0.7142857142857143
f1_score 0.6666666666666666


In [35]:
import itertools
from itertools import product 

In [36]:
result = itertools.combinations(X ,2)

for item in result:
    df  = pd.DataFrame(item)
    X[list (item)]
    parameters = {
    'learning_rate': [0.01 , 0.1 ,1],
    'n_estimators': [12,14,16],
    'subsample' : [0.7],
    'max_depth' : [2] ,
    'random_state':[2]
    }
    GradientBoosting = GradientBoostingClassifier()
    clf = GridSearchCV(GradientBoosting , parameters , cv=3 , scoring='f1_macro')
    clf.fit(X_train[list (item)],y_train)
    clf.predict(X_test[list (item)])
    #print('predict=',clf.predict(X_test[list (item)]))
    #print(X_test)
    #print("y_train = ",y_train)
    print(item)
    print("precision_score =",precision_score(clf.predict(X[list(item)]),y))
    print("recall_score =", recall_score(clf.predict(X[list(item)]),y))
    print("f_1=",f1_score(clf.predict(X[list(item)]),y,average='macro'))
    print("f_1_X-test=",f1_score(clf.predict(X_test[list(item)]),y_test,average='macro'))

('commit_befor ', 'commit_after')
precision_score = 0.5882352941176471
recall_score = 0.9090909090909091
f_1= 0.7321428571428571
f_1_X-test= 0.375
('commit_befor ', 'cal_smell')
precision_score = 0.5882352941176471
recall_score = 0.8333333333333334
f_1= 0.699666295884316
f_1_X-test= 0.4949494949494949
('commit_befor ', 'dispen_bf')
precision_score = 0.7058823529411765
recall_score = 0.8
f_1= 0.7321428571428572
f_1_X-test= 0.5833333333333333
('commit_befor ', 'object_bf')
precision_score = 0.4117647058823529
recall_score = 1.0
f_1= 0.6527777777777778
f_1_X-test= 0.23076923076923075
('commit_befor ', 'bloater_bf')
precision_score = 0.8235294117647058
recall_score = 0.7
f_1= 0.6827262044653348
f_1_X-test= 0.6703296703296704
('commit_befor ', 'dispen_at')
precision_score = 0.47058823529411764
recall_score = 0.8888888888888888
f_1= 0.6606334841628959
f_1_X-test= 0.375
('commit_befor ', 'object_at')
precision_score = 0.5294117647058824
recall_score = 0.9
f_1= 0.696969696969697
f_1_X-test= 0.

('object_bf', 'changed_file')
precision_score = 0.8235294117647058
recall_score = 0.9333333333333333
f_1= 0.8660714285714285
f_1_X-test= 0.5833333333333333
('object_bf', 'additions')
precision_score = 0.6470588235294118
recall_score = 0.7857142857142857
f_1= 0.6996662958843158
f_1_X-test= 0.5833333333333333
('object_bf', 'deletions')
precision_score = 0.8235294117647058
recall_score = 0.7777777777777778
f_1= 0.7599999999999999
f_1_X-test= 0.7619047619047619
('object_bf', 'dev')
precision_score = 0.11764705882352941
recall_score = 1.0
f_1= 0.42233632862644416
f_1_X-test= 0.23076923076923075
('bloater_bf', 'dispen_at')
precision_score = 0.4117647058823529
recall_score = 1.0
f_1= 0.6527777777777778
f_1_X-test= 0.4949494949494949
('bloater_bf', 'object_at')
precision_score = 0.8235294117647058
recall_score = 0.7
f_1= 0.6827262044653348
f_1_X-test= 0.6703296703296704
('bloater_bf', 'bloater_af')
precision_score = 0.8235294117647058
recall_score = 0.7777777777777778
f_1= 0.7599999999999999
f